# Setup

In [4]:
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
import utils as U
reload(U)
import image_loader as IM
reload(IM)
from theano.sandbox import cuda
from keras.preprocessing import image
%matplotlib inline
import os
from keras.models import Model
from keras.layers import Dense, BatchNormalization, Input, Flatten, Conv2D, AveragePooling2D
from keras.optimizers import Adam
import models.model as M
from keras.layers import Flatten, Dense, BatchNormalization
from keras.models import Sequential
from keras.layers import Dropout
from keras.regularizers import l2
reload(M)

<module 'models.model' from 'models/model.pyc'>

In [5]:
folder = "data/train/"
train_folder = folder+"train"
train_filenames = os.listdir(train_folder)
validation_folder = folder+"valid"
validation_filenames = os.listdir(validation_folder)
batch_size = 44
train_batch_count = len(train_filenames) / batch_size
val_batch_count = len(validation_filenames) / batch_size

mapping = U.atmospheric_mapping()

train_labels = [ mapping[fname] for fname in train_filenames ]
valid_labels = [ mapping[fname] for fname in validation_filenames ]

In [6]:
gen_t = image.ImageDataGenerator(vertical_flip=True, horizontal_flip=True)

gen = IM.DirectoryIterator(train_folder, train_filenames, image_processor=gen_t, one_hot_filename_mapping=mapping, batch_size=batch_size, shuffle=True)
val_gen = IM.DirectoryIterator(validation_folder, validation_filenames, one_hot_filename_mapping=mapping, batch_size=batch_size, shuffle=False)

# Pre compute conv layers

In [3]:
conv_model = M.full_model()
conv_model.load_weights("weights/atmospheric-full-training.07-1.78.hdf5")
conv_model = Sequential(conv_model.layers[:-len(M.dense_layers())])

conv_model.compile(optimizer=(Adam(lr=0.0001)), metrics=["accuracy"], loss="categorical_crossentropy")
gen = IM.DirectoryIterator(train_folder, train_filenames, image_processor=gen_t, one_hot_filename_mapping=mapping, batch_size=batch_size, shuffle=False)
val_gen = IM.DirectoryIterator(validation_folder, validation_filenames, one_hot_filename_mapping=mapping, batch_size=batch_size, shuffle=False)
train_conv_features = conv_model.predict_generator(gen, train_batch_count, verbose=1)
val_conv_features = conv_model.predict_generator(val_gen, val_batch_count, verbose=1)


NameError: name 'gen_t' is not defined

# Dense Training

In [9]:
dense_model = M.full_model(dropout=0.3, regularization=0.001)
dense_model.load_weights("weights/atmospheric-full-training.07-1.78.hdf5")
flat = Flatten(input_shape=(8,8,512))
dense_model = Sequential([flat] + dense_model.layers[-len(M.dense_layers()) + 1:])

dense_model.compile(optimizer=(Adam(lr=0.001)), metrics=["accuracy"], loss="categorical_crossentropy")

train_conv_atmospheric_features = U.load_array("train_conv_atmospheric_features")
val_conv_atmospheric_features = U.load_array("valid_conv_atmospheric_features")


In [7]:
weights = { 0: 13.6, 1: 3.9, 2: 10.5, 3: 1 }
dense_model.fit(train_conv_atmospheric_features, train_labels, class_weight=weights, batch_size=64, validation_data=(val_conv_atmospheric_features, valid_labels), epochs=4)

Train on 34816 samples, validate on 5632 samples
Epoch 1/4
34816/34816 [==============================] - 72s - loss: 1.8089 - acc: 0.8215 - val_loss: 0.7651 - val_acc: 0.7562
Epoch 2/4
34816/34816 [==============================] - 74s - loss: 1.6418 - acc: 0.8321 - val_loss: 0.4349 - val_acc: 0.8473
Epoch 3/4
34816/34816 [==============================] - 83s - loss: 1.5730 - acc: 0.8378 - val_loss: 0.5762 - val_acc: 0.8191
Epoch 4/4
34816/34816 [==============================] - 83s - loss: 1.6045 - acc: 0.8325 - val_loss: 0.4445 - val_acc: 0.8542


In [10]:
weights = { 0: 13.6, 1: 3.9, 2: 10.5, 3: 1 }
dense_model.fit(train_conv_atmospheric_features, train_labels, class_weight=weights, batch_size=64, validation_data=(val_conv_atmospheric_features, valid_labels), epochs=6)

Train on 34816 samples, validate on 5632 samples
Epoch 1/6
34816/34816 [==============================] - 245s - loss: 3.6899 - acc: 0.8122 - val_loss: 2.6995 - val_acc: 0.8226
Epoch 2/6
34816/34816 [==============================] - 244s - loss: 4.7218 - acc: 0.8150 - val_loss: 3.3845 - val_acc: 0.8331
Epoch 3/6
34816/34816 [==============================] - 244s - loss: 4.8040 - acc: 0.8206 - val_loss: 2.9306 - val_acc: 0.8606
Epoch 4/6
34816/34816 [==============================] - 244s - loss: 5.1760 - acc: 0.8161 - val_loss: 3.3343 - val_acc: 0.8640
Epoch 5/6
34816/34816 [==============================] - 244s - loss: 5.0445 - acc: 0.8175 - val_loss: 3.4123 - val_acc: 0.8100
Epoch 6/6
34816/34816 [==============================] - 244s - loss: 5.0092 - acc: 0.8184 - val_loss: 2.9689 - val_acc: 0.8427


# Full Training

In [ ]:
model = M.full_model()
model.load_weights("weights/atmospheric-full-training.07-1.78.hdf5")

weights = { 0: 13.6, 1: 3.9, 2: 10.5, 3: 1 }
model.compile(optimizer=(Adam(lr=0.0001)), metrics=["accuracy"], loss="categorical_crossentropy")
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, BaseLogger, History, CSVLogger
callbacks = [
    History(),
    BaseLogger(),
    ModelCheckpoint("weights/3-a-atmospheric-full-training.{epoch:02d}-{val_loss:.2f}.hdf5", monitor="val_loss", verbose=0, save_best_only=False, save_weights_only=True, mode='auto', period=1),
    CSVLogger("results/atmospheric_3.csv", separator=',', append=False)
]

 
model.fit_generator(gen, train_batch_count, class_weight=weights, epochs=4, validation_data=val_gen, validation_steps=val_batch_count, callbacks = callbacks)

Epoch 1/4
276/791 [=========>....................] - ETA: 3475s - loss: 1.4952 - acc: 0.8412

In [ ]:
model.fit_generator(gen, train_batch_count, class_weight=weights, epochs=14, validation_data=val_gen, validation_steps=val_batch_count, callbacks = callbacks)